In [ ]:
import requests
from datetime import datetime

# Load configuration (same as your main script)
from config import get_config
config = get_config()

GITHUB_TOKEN = config['GITHUB_TOKEN']
GITHUB_REPO = config['GITHUB_REPO']

def list_blog_posts():
    """List all blog posts in the _posts directory"""
    url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/_posts"
    headers = {
        "Authorization": f"token {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json"
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        posts = response.json()
        blog_posts = []
        
        for post in posts:
            if post['name'].endswith('.md'):
                blog_posts.append({
                    'name': post['name'],
                    'path': post['path'],
                    'sha': post['sha'],
                    'download_url': post['download_url']
                })
        
        # Sort by name (which includes date) - newest first
        blog_posts.sort(key=lambda x: x['name'], reverse=True)
        return blog_posts
        
    except requests.RequestException as e:
        print(f"Error fetching blog posts: {e}")
        return []

def delete_blog_post(post_info):
    """Delete a single blog post"""
    url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/{post_info['path']}"
    headers = {
        "Authorization": f"token {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json"
    }
    
    data = {
        "message": f"Delete blog post: {post_info['name']}",
        "sha": post_info['sha']
    }
    
    try:
        response = requests.delete(url, headers=headers, json=data)
        response.raise_for_status()
        print(f"✅ Deleted: {post_info['name']}")
        return True
        
    except requests.RequestException as e:
        print(f"❌ Failed to delete {post_info['name']}: {e}")
        return False

def delete_latest_blogs(num_to_delete, confirm=True):
    """
    Delete the most recent blog posts
    
    Args:
        num_to_delete (int): Number of blog posts to delete
        confirm (bool): Whether to ask for confirmation before deleting
    """
    print(f"Fetching list of blog posts...")
    posts = list_blog_posts()
    
    if not posts:
        print("No blog posts found.")
        return
    
    if num_to_delete > len(posts):
        print(f"You asked to delete {num_to_delete} posts, but only {len(posts)} exist.")
        num_to_delete = len(posts)
    
    posts_to_delete = posts[:num_to_delete]
    
    print(f"\nFound {len(posts)} total blog posts.")
    print(f"Will delete the {num_to_delete} most recent posts:")
    print("-" * 50)
    
    for i, post in enumerate(posts_to_delete, 1):
        print(f"{i}. {post['name']}")
    
    if confirm:
        print("-" * 50)
        confirmation = input(f"\nAre you sure you want to delete these {num_to_delete} posts? (yes/no): ")
        if confirmation.lower() not in ['yes', 'y']:
            print("Deletion cancelled.")
            return
    
    print(f"\nDeleting {num_to_delete} blog posts...")
    deleted_count = 0
    
    for post in posts_to_delete:
        if delete_blog_post(post):
            deleted_count += 1
    
    print(f"\n✅ Successfully deleted {deleted_count} out of {num_to_delete} blog posts.")

def delete_blogs_by_date(date_string):
    """
    Delete all blog posts from a specific date
    
    Args:
        date_string (str): Date in format "2025-05-27"
    """
    print(f"Fetching blog posts from {date_string}...")
    posts = list_blog_posts()
    
    posts_to_delete = [post for post in posts if post['name'].startswith(date_string)]
    
    if not posts_to_delete:
        print(f"No blog posts found for date {date_string}")
        return
    
    print(f"Found {len(posts_to_delete)} blog posts from {date_string}:")
    for post in posts_to_delete:
        print(f"  - {post['name']}")
    
    confirmation = input(f"\nDelete all {len(posts_to_delete)} posts from {date_string}? (yes/no): ")
    if confirmation.lower() not in ['yes', 'y']:
        print("Deletion cancelled.")
        return
    
    deleted_count = 0
    for post in posts_to_delete:
        if delete_blog_post(post):
            deleted_count += 1
    
    print(f"\n✅ Successfully deleted {deleted_count} out of {len(posts_to_delete)} blog posts.")

def show_all_posts():
    """Display all blog posts with numbers for easy reference"""
    posts = list_blog_posts()
    
    if not posts:
        print("No blog posts found.")
        return
    
    print(f"Found {len(posts)} blog posts:")
    print("-" * 70)
    
    for i, post in enumerate(posts, 1):
        print(f"{i:2d}. {post['name']}")
    
    return posts

# Usage examples
if __name__ == "__main__":
    print("BLOG POST DELETION TOOL")
    print("=" * 50)
    
    while True:
        print("\nOptions:")
        print("1. Show all blog posts")
        print("2. Delete latest N posts")
        print("3. Delete posts by date")
        print("4. Exit")
        
        choice = input("\nEnter your choice (1-4): ").strip()
        
        if choice == "1":
            show_all_posts()
            
        elif choice == "2":
            try:
                num = int(input("How many recent posts to delete? "))
                delete_latest_blogs(num)
            except ValueError:
                print("Please enter a valid number.")
                
        elif choice == "3":
            date_str = input("Enter date (YYYY-MM-DD): ").strip()
            delete_blogs_by_date(date_str)
            
        elif choice == "4":
            print("Goodbye!")
            break
            
        else:
            print("Invalid choice. Please try again.")